# Beer Hops Data: Exploratory Data Analysis & Summary Analysis

**Data Files:** *cln_hops_profile.csv, cln_hops_brewvalues.csv*

**Original Source:** *https://beermaverick.com/hops/*  (Data retrieved via web-scraping)

------------------------------------------------------------

Define libraries:

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import geopandas as gpd
import seaborn as sns
import pandas as pd
import numpy as np
import folium

Define file paths for processed CSV data:

In [ ]:
CLEAN_HOPS_PATH = './clean_data/cln_hops_brewvalues.csv'
CLEAN_HOPS_PROFILE_PATH = './clean_data/cln_hops_profile.csv'

Read in clean CSV data into local dataframes with index as the Hop Name:

In [ ]:
hops_v = pd.read_csv(CLEAN_HOPS_PATH, index_col='Hop Name')
hops_p = pd.read_csv(CLEAN_HOPS_PROFILE_PATH, index_col='Hop Name')

Combine dataframes to have all the hop data in a master dataframe:

In [ ]:
hops_main = pd.concat([hops_v, hops_p], axis=1)
hops_main.head()

Shorten the country 'United States of America' to 'USA'. This will look cleaner on plots.

In [ ]:
hops_main['Country'].replace("United States of America", "USA", inplace=True)

Make a new column for Continent:

In [ ]:
hops_main['Continent'] = hops_main['Country'].copy()
hops_main['Continent'].replace({
    'Australia': 'Australia',
    'Canada': 'N. America',
    'China': 'Asia',
    'Czech Republic': 'Europe',
    'France': 'Europe',
    'Germany': 'Europe',
    'Japan': 'Asia',
    'New Zealand': 'Australia',
    'Poland': 'Europe',
    'Slovenia': 'Europe',
    'South Africa': 'Africa',
    'Ukraine': 'Europe',
    'United Kingdom': 'Europe',
    'USA': 'N. America'},
    inplace=True)

hops_main.Continent.unique() 

## Exploratory Data Analysis
### Brewing Value Analysis

Select only the brewing value average columns + 'Purpose', 'Country', and 'Continent'.

In [ ]:
hops_bv = pd.DataFrame(hops_main, columns=[
    'Alpha Acid % - Avg',
    'Beta Acid % - Avg',
    'Alpha-Beta Ratio - Avg',
    'Co-Humulone as % of Alpha - Avg',
    'Total Oils (mL/100g) - Avg',
    'Myrcene - Avg',
    'Humulene - Avg',
    'Caryophyllene - Avg',
    'Farnesene - Avg',
    'Purpose',
    'Country',
    'Continent'])

hops_bv.head()

Inspect the features of the dataframe:

In [ ]:
hops_bv.info()

Check for null values:

In [ ]:
hops_bv.isnull().sum()

There are some missing values that were converted into np.nan in previous steps.These must be remove for visualizations and analysis. Dataset without NAs:

In [ ]:
hops_bv = hops_bv.dropna(axis=0)
hops_bv.sample(5)

Plot of the number of hops per country with Purpose breakdown:

In [ ]:
fig = px.bar(hops_bv,
             x="Country",
             color="Purpose",
             labels={
                 "x": "Country", "count": "Count"},
             title="Number of Hops per Country",
             hover_data=['Continent'],
             width=800,
             height=1000)
fig.update_layout(title={
    'y': 0.95,
    'x': 0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    xaxis={
        'categoryorder': 'total descending'})
# fig.write_image("images/HopsPerCountry.png")
fig.show()

This graphic shows the count of hops per country in the dataset. Quite clearly, the United States of America has developed the most hops out of all the countries. Followed by Germany and New Zealand. Europe as a whole would come in second after USA. The colors in the plot represent the use purpose of the hop. Some hops are used specifically for bittering, while others are used more for aromas. Some hops give the best of both worlds and have dual use. Countries like USA and New Zealand have mostly dual-use hops, while European countries such as Germany and Czech Republic have mostly aromatic hops. As hop techniques improve and the focus from using hops as a bittering agent has shifted towards it’s aromatic and flavorful offerings. As it can be seen in the graphic, most hops have either a dual purpose or aromatic purpose.

The last plot suggested that it may be better to view the the plot but per Continent instead of country.Hovering over the bars will show the exact count.

In [ ]:
count_cont = hops_bv['Continent'].value_counts()
count_cont

In [ ]:
fig = px.bar(count_cont, title='Hops per Continent', labels={'value':'Count', 'index':'Continent'}, color=count_cont.index)
fig.update_layout(title={
    'y': 0.9,
    'x': 0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    xaxis={
        'categoryorder': 'total descending'},
        showlegend=False)

# fig.write_image("images/HopsPerContinent.png")
fig.show()

The following violin and box whisker plots are interactive. Use the mouse to hover over the plot to read the values of minimum, maximum, median, and quantiles.

In [ ]:
for i in hops_bv.columns[:9]:
    fig = px.violin(hops_bv, y=i, height=500, width=500)
    fig.show()
#     if i.startswith("Total"):
#         fig.write_image("images/Violin_TotalOil.png")
#     else:
#         fig.write_image(f"images/Violin_{i}.png")

Box-whisker Plots

In [ ]:
for i in hops_bv.columns[:9]:
    fig = px.box(hops_bv, y=i, height=500, width=500)
    fig.show()
#     if i.startswith("Total"):
#         fig.write_image("images/Box_TotalOil.png")
#     else:
#         fig.write_image(f"images/Box_{i}.png")

In observing the violin and box plots above, there is a minimal amount of outliers. The average Farnesene feature seems to have the most significant amount of outliers.  

In order to study the correlation of the variables to one another, a heatmap is created.

In [ ]:
plt.figure(dpi=150)
fig = sns.heatmap(hops_bv[:9].corr(), cmap='coolwarm', vmax=1,
            vmin=-1, xticklabels=True, yticklabels=True)
plt.title("Correlation Hop Brewing Values & Oil Concentrations")
plt.xticks(rotation=90, fontsize=7)
plt.yticks(fontsize=7)
# plt.savefig('images/heatmap.png', dpi=200, bbox_inches = "tight")
plt.show()


Using .describe() the dataset to inspect some importants EDA values such as minimum, maximum, mean, strandard deviation and quartiles.

In [ ]:
hops_bv.describe()

The following are two pair plots - one with 'Purpose' as the color of the bars, and one with 'Continent' as the bar colors.

To get the darta xonvert 'inf' to np.nan in column 'Alpha-Beta Ratio - Avg' and remove rows with it

Pairplot with Purpose as the color

In [ ]:
fig = sns.pairplot(hops_bv, height=3, hue='Purpose')
# fig.savefig('images/Pairplot_purpose.png') 

From this plot, it is most notable the Alpha Acid percentage and the alpha-beta ratio are the most significantly different from one in other regarding purpose. The peaks of the colored curves in the top left plot are significantly different with a bit of an overlap between bittering hops and dual purpose hops. This makes sense because dual purpose hops have the best of both worlds - aroma and bitter properties.

In the first row of plots, it can be observed that the aromatic hops (in orange) are somewhat clustered together as compare to the bittering and dual purpose hops, which are mostly layed over eachother.

Pairplot with Continent as the color

In [ ]:
fig = sns.pairplot(hops_bv, height=3, hue='Continent')
# fig.savefig('images/Pairplot_country.png') 

From the plot it can be seen that some variables are more correlated than others. A strong positive correlation is indicated by the darker red squares, while a strong positive correlation is indicated by the darker blue. The most signifcant positive correlations include:
- 'Co-Humulone as % of Alpha - Avg' with 'Alpha Acid % - Avg'
- 'Co-Humulone as % of Alpha - Avg' with 'Alpha-Beta Ratio - Avg'
- 'Farnesene - Avg' with 'Alpha-Beta Ratio - Avg'

The most significant negative correlations include:
- 'Co-Humulone as % of Alpha - Avg' with 'Humulene - Avg'
- 'Alpha Acid % - Avg' with 'Humulene - Avg'
- Crayophyllene - Avg, with Myrcene

## Summary Visuals & Analysis

<!-- hops_bv['Alpha-Beta Ratio - Avg'].replace(np.inf, np.nan, inplace=True)
hops_bv[ hops_bv['Alpha-Beta Ratio - Avg'] == np.inf]
hops_bv = hops_bv.dropna(axis=0) -->

### Alpha Acids and Bitterness

Alpha acids (α acids) are a class of chemical compounds found in the resin glands of the hop plant flowers. They are the source of bitterness in beer, and they possess anti-bacterial properties. The bitterness level is a result of a process called isomerization which happens in the boiling stage of the brewing process. The degree of isomerization and hence the bitterness are highly dependent on the length of time the hops are boiled. Longer boil times result in isomerization of more alpha acids and therefore increase the bitterness. 

In [ ]:
fig = px.bar(hops_bv, y='Alpha Acid % - Avg', orientation='v',
             color='Purpose', title='Bitterness in Hops', height=800, hover_data=['Continent'])
fig.update_layout(title={
    'y': 0.9,
    'x': 0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    xaxis={'categoryorder': 'total ascending'})
# fig.write_image("images/BitternessInHops.png")    
fig.show()


In plot above, the height of the bars represent the alpha acids % averages of each hop. The color of the bar represents the Purpose of the hop. As expected, the hops with lower alpha acid % (on the left side of the x-axis) are mostly for aromatic purposes. On the other side, with the higher alpha acid %, we have mostly dual purpose and bittering hops. This supports the notion that the acid percentages in the hop can affect it's purpose.

To further inspect this, the dataframe is grouped by 'Purpose', then each  group is averaged and the results are plotted.

In [ ]:
hops_bv_purpose = hops_bv.groupby("Purpose").mean()

hops_bv_purpose

The average alpha acid % is significantly lower in the aromatic hops, as compared to the bittering and dual purpose hops. This make sense as to why the are labelled as such.

In [ ]:
fig = px.bar(hops_bv_purpose, y="Alpha Acid % - Avg", orientation='v',
             color=hops_bv_purpose.index, width=500, height=500,
             title="Alpha Acids and Bitterness")
fig.update_layout(title={
    'y': 0.9,
    'x': 0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    xaxis={'categoryorder': 'total ascending'})
# fig.write_image("images/BitternessInHops.png")        
fig.show()


### Aroma Tag Analysis

Make a dataframe consisting of the aroma tags, 'Country', 'Purpose', and 'Continent'. Then drop rows with NAs:

In [ ]:
hops_tags = hops_p.copy()
hops_tags['Continent'] = hops_tags['Country'].copy()
hops_tags['Continent'].replace({
    'Australia': 'Australia',
    'Canada': 'N. America',
    'China': 'Asia',
    'Czech Republic': 'Europe',
    'France': 'Europe',
    'Germany': 'Europe',
    'Japan': 'Asia',
    'New Zealand': 'Australia',
    'Poland': 'Europe',
    'Slovenia': 'Europe',
    'South Africa': 'Africa',
    'Ukraine': 'Europe',
    'United Kingdom': 'Europe',
    'United States of America': 'N. America'},
    inplace=True)

hops_tags = hops_tags.dropna(axis=0)

hops_tags.head()


In order to calculate the most used tags for each continent, the dataframe hops_tag is grouped by 'Continent' and saved as a new dataframe hops_tag_g. The False/True are replaced by 0/1 and the columns are sumed. 

In [ ]:
hops_tags_g = hops_tags.copy()
hops_tags_g = hops_tags_g.replace({False:0, True:1})
hops_tags_g = hops_tags_g.groupby('Continent').sum()
hops_tags_g

For each continent, the 5 most used aroma tags are printed and plotted:

In [ ]:
for i in hops_tags_g.index:
    # print(hops_tags_g[hops_tags_g.index == i])
    count = hops_tags_g[hops_tags_g.index == i].copy().sum()
    count = count.astype('int32')
    count = count.nlargest(n=5, keep='first')
    print(f"5 most used Aroma tags for {i}:")
    print(", ".join(count.index))
    print()
    fig = px.bar(count, title=i, color=count.index)
#     fig.write_image(f"images/MostUsedHops_{i}.png")    
    fig.show()

An alternative approach to calculate the most used aroma tags per continent is presented below. 

Inspect the the value counts in hops_tags. Drop columns that have less than 20 True entries. This leaves a dataset with the 21 most used aroma profiles.

In [ ]:
# TOP 21

for i in hops_tags.columns[2:-1]:
    if hops_tags[i].value_counts()[True] < 20:
        hops_tags = hops_tags.drop([i], axis=1)
hops_tags


Create a bar plot to display the 21 most used aroma profiles.

In [ ]:
aroma_count = hops_tags.sum()

fig = px.bar(x=aroma_count.index[2:-1], y=aroma_count.values[2:-1],
             title='21 Most Popular Aroma Tags', height=750, color=aroma_count.index[2:-1], 
             labels={
             "x": "Aroma Tag",
             "y": "Count"
             })

fig.update_layout(title={
    'y': 0.9,
    'x': 0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    xaxis={'categoryorder': 'total descending'})
# fig.write_image("images/21MostUsedHops.png")        
fig.show()


Further cut down the dataset to only have the 6 most used aroma profiles: citrus, floral, spicy, pine, herbal, and fruity.

In [ ]:
# TOP 6

for i in hops_tags.columns[2:-1]:
    if hops_tags[i].value_counts()[True] < 50:
        hops_tags = hops_tags.drop([i], axis=1)

hops_tags

Group hops_tags dataframe by Continent.

In [ ]:
hops_tags_group = hops_tags.groupby("Continent").sum()


Drop the columns 'Purpose' and 'Country'. Also drop the gouping for 'Asia' since it has very little entries.

In [ ]:
hops_tags_group = hops_tags_group.drop(['Purpose', 'Country'], axis=1)
hops_tags_group = hops_tags_group.drop(['Asia'], axis=0)
hops_tags_group


Use the grouped dataframe to plot the breakdown of the 6 most used hop aromas in each continent.

In [ ]:
fig = px.bar(hops_tags_group, title="6 Most Popular Aromas - Breakdown per Continent", width=600, height=750, labels={
    "value": "Count", "variable": "Aroma Tag"})
fig.update_layout(title={
    'y': 0.9,
    'x': 0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    xaxis={'categoryorder': 'total ascending'})
# fig.write_image("images/AromasPerContinent.png")
fig.show()


Create a smaller dataframe from hops_bv to only include the Oil Concentration Averages of each hop and the Continent

In [ ]:
hops_bv_sm = hops_bv.drop(['Purpose', 'Country'], axis=1)

hops_bv_sm = hops_bv_sm.drop(['Alpha Acid % - Avg', 'Beta Acid % - Avg', 'Alpha-Beta Ratio - Avg',
            'Co-Humulone as % of Alpha - Avg', 'Total Oils (mL/100g) - Avg'], axis=1).copy()
hops_bv_sm.head()


Breakdown of the oil concentration in each hop. Click on the oil in the legend to show/hide.

Myrcene and Humulene are the most present oils in the hops. These are responsible for 

In [ ]:
fig = px.bar(hops_bv_sm, orientation='h', width=750,
             height=1500, hover_data=['Continent'],
             title='Oil Concentration in Hops',
             labels={'variable':'Oils', 'value':'Total Oil Breakdown (%)'}
             )
fig.update_layout(title={
    'y': .95,
    'x': 0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    yaxis={'categoryorder': 'total descending'})
# fig.write_image("images/OilsInHops.png")
fig.show()


Same process but this time for breakdown of the acid concentration in each hop. Click on the oil in the legend to show/hide.

In [ ]:
hops_bv_sm2 = hops_bv.copy()
hops_bv_sm2 = hops_bv_sm2.drop(['Myrcene - Avg', 'Humulene - Avg', 'Caryophyllene - Avg',
            'Farnesene - Avg', 'Total Oils (mL/100g) - Avg', 'Total Oils (mL/100g) - Avg', 'Alpha-Beta Ratio - Avg',
            'Co-Humulone as % of Alpha - Avg'], axis=1).copy()
hops_bv_sm2            

In [ ]:
fig = px.bar(hops_bv_sm2, orientation='h', width=750,
             height=1500, hover_data=['Continent', 'Purpose', 'Country'],
             title='Acids in Hops',
             labels={'variable':'Acids', 'value':'Acid %'}
             )
fig.update_layout(title={
    'y': .95,
    'x': 0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    yaxis={'categoryorder': 'total descending'})
# fig.write_image("images/AcidsInHops.png")
fig.show()

In order to explore the oils for the aroma tags, the following interactive plots were created. Click the oils in the legend to show or hide the corresponding oils.

In [ ]:
hops_group = hops_bv_sm.groupby('Continent').mean()
hops_group1 = hops_group.drop(['Asia', 'Australia', 'Europe'])
hops_group2 = hops_group.drop(['N. America', 'Asia', 'Africa'])

fig1 = px.bar(hops_group1, width=500, height=500,
              title="Average Oil Concentration",
              labels={'variable':'Oils', 'value':'Oil %'})
fig1.update_layout(title={
    'y': .9,
    'x': 0.5,
    'xanchor': 'center',
    'yanchor': 'top'})
fig1.show()

While both North America and South Africa share citrus and floral as their top two, the rest of the tags are different. North American hops have spicy, pine, and grapefruit which are not in the top five of South African hops. On the other hand, South African hops are described to have herbal, lemongrass, and berry aroma tags which are not present in the American top five.

Myrcene is mostly associated with citrusy and resinous-pine aromas. North American hops have pine in their top 5 while South African do not. The expectation is that North American hops have a higher myrcene percentages as compared to South African hops. Based on this data, they do. It can be observed in the plot to the left.

Higher humulene concentrations lend to floral, herbal, and black pepper(spicy) aromas, which aligns more so with South African hops than North American hops. The expectation is the South African hops would have higher percentages of humulene. Based on this date, it does. I can be seen in the plot to the right.

Caryophyllene is mostly associated with black pepper, spiciness, and herbal aromas. This is also aligned more with the top 5 South African aromas tags. As expected, the plot to the right shows that the average percentages of Caryophyllene are higher in South African hops than North American.

In [ ]:
fig2 = px.bar(hops_group2, width=500, height=500,
              title="Average Oil Concentration",
              labels={'variable':'Oils', 'value':'Oil %'})
fig2.update_layout(title={
    'y': .9,
    'x': 0.5,
    'xanchor': 'center',
    'yanchor': 'top'})
fig2.show()

The aroma tags most used to describe Australian and New Zealand are citrus, tropical fruit, pine, passion fruit, and floral. This is quite different than the aroma tags most used to describe European hops, which are floral, spicy, citrus, herbal, and fruity. The only tags shared by both continents are citrus and floral. While citrus is the most used in Australia, it is third in Europe. European hops are most described as floral, which is the fifth most used tag in Australian and New Zealand hops. Some European hops are described as spicy, herbal, and fruity, which do not appear in the Australian and New Zealand top five. On the other hand, Australian and New Zealand hops are described to have passion fruit, tropical fruit, and pine, which are not present in the European top five aromas.

Humulene is mostly associated with woody and pine aromas; however, hops with higher amounts tend to be more floral, herbal, and black pepper(spicy) in character. These tags align much more with the European top 5, than the Australian top 5.

### Choropleths: Geographical Summary of Hops Data

Read in geojson data with filtered countries to be used for Folium mapping.

In [ ]:
countries_df = gpd.read_file('./raw_data/countries.geojson')
countries_df = countries_df[countries_df.ADMIN.isin(hops_p.Country.unique())]
countries_df.reset_index(drop=True, inplace=True)
countries_df

Retrieve country info for values to be used for visualization and add average brew values for each into geojson dataframe.

In [ ]:
values_countries = hops_v.merge(hops_p.Country, left_index=True, right_index=True)
values_countries.replace([np.inf, -np.inf], np.nan, inplace=True)  # replacing inf values

for col in [i for i in values_countries.columns if 'Avg' in i]:  # avg value columns only
    mean_values_per_country = values_countries.groupby(by='Country', dropna=True).mean()[col]
    countries_df[col] = mean_values_per_country.values

# Adding popular aromas for hops for each country
country_list = sorted(list(hops_p.Country.unique()))
aromas_lists = []
for i in range(len(country_list)):
    df = hops_p[hops_p.Country == country_list[i]].copy()
    df.drop(columns=['Purpose', 'Country'], inplace=True)
    aroma_count = df.sum()
    aroma_count = aroma_count.astype('int32')
    aroma_count = aroma_count.nlargest(n=3, keep='first')
    aromas_lists.append(list(aroma_count.index))
countries_df['Top 3 Aromas'] = aromas_lists

countries_df

Output the choropleth visual. For better viewing, refer to the 'images/choro_values.html' in the repository.

In [ ]:
m = folium.Map(location=[0, 0], zoom_start=2)  # starting point

def create_choro(brew_value):
    """Sets up choropleth specifications for a given brew value and returns the choropleth object."""
    choro = folium.Choropleth(
        geo_data=countries_df,
        name=brew_value,
        data=countries_df,
        columns=['ADMIN', brew_value],
        key_on='feature.properties.ADMIN',
        fill_color='YlOrRd',
        fill_opacity=0.8,
        line_opacity=.2,
        line_weight=2,
        smooth_factor=0,
        Highlight=True,
        nan_fill_color='White',
        legend_name=f'Brew Value: {brew_value}',
        show=False,
        highlight=True,
        overlay=False
    )
    return choro

# Add choropleth layer for each brew value
for brew_value in [i for i in values_countries.columns if 'Avg' in i]:
    m.add_child(create_choro(brew_value))

# Add information markers for popular aromas
centers = countries_df.to_crs('+proj=cea').centroid.to_crs(countries_df.crs)
for i in range(len(countries_df)):
    m.add_child(
        folium.Marker(
            location=[centers[i].y, centers[i].x],
            popup=f"""
                <p>Top 3 Aromas:</p> <hr>
                <p>{countries_df.iloc[i]["Top 3 Aromas"][0]}</p> <hr>
                <p>{countries_df.iloc[i]["Top 3 Aromas"][1]}</p> <hr>
                <p>{countries_df.iloc[i]["Top 3 Aromas"][2]}</p> <hr>
            """,
            icon=folium.Icon(color='blue')
        )
    )

m.add_child(folium.LayerControl(position='topright', collapsed=False))
# m.save('./images/choro_values.html')
m